In [3]:
from PIL import Image, ImageDraw, ImageFont
from PyPDF2 import PdfMerger
import os

In [4]:
def create_subject_roi_event_pdf(sub, roi):
    
    ## create file list
    rivendel_dir = '/home/brooke/pacman/preprocessing'
    image_paths = []
    image_paths.append(f'{rivendel_dir}/{sub}/scripts/figures/{sub}_average_{roi}_trialonset.png')
    image_paths.append(f'{rivendel_dir}/{sub}/scripts/figures/{sub}_average_{roi}_firstmove.png')
    image_paths.append(f'{rivendel_dir}/{sub}/scripts/figures/{sub}_average_{roi}_firstdot.png')
    image_paths.append(f'{rivendel_dir}/{sub}/scripts/figures/{sub}_average_{roi}_conflict.png')
    image_paths.append(f'{rivendel_dir}/{sub}/scripts/figures/{sub}_average_{roi}_noconflict.png')
    image_paths.append(f'{rivendel_dir}/{sub}/scripts/figures/{sub}_average_{roi}_ghostattack.png')
    image_paths.append(f'{rivendel_dir}/{sub}/scripts/figures/{sub}_average_{roi}_escape.png')
    image_paths.append(f'{rivendel_dir}/{sub}/scripts/figures/{sub}_average_{roi}_chase.png')
    image_paths.append(f'{rivendel_dir}/{sub}/scripts/figures/{sub}_average_{roi}_died.png')
    
    ## creste pdf 
    pdf_width_in = 20  # in inches
    pdf_height_in = 2.5  # in inches

    image_width_in = 2.5  # target image width in inches

    pdf_width_px = int(pdf_width_in * 300)  # width in pixels, using 300 DPI
    pdf_height_px = int(pdf_height_in * 300)  # height in pixels, using 300 DPI
    image_width_px = int(image_width_in * 300)  # image width in pixels, using 300 DPI

    # Start with a blank white "canvas" for the PDF
    pdf_img = Image.new('RGB', (pdf_width_px, pdf_height_px), 'white')

    for i, img_path in enumerate(image_paths):
        img = Image.open(img_path)

        # If the image has an alpha channel, composite it over a white background
        if img.mode == 'RGBA':
            white_bg = Image.new('RGBA', img.size, (255, 255, 255, 255))  # full opacity
            img = Image.alpha_composite(white_bg, img)

        img = img.convert('RGB')

        # Resize the image while maintaining its aspect ratio
        wpercent = (image_width_px / float(img.size[0]))
        hsize = int((float(img.size[1]) * float(wpercent)))
        img = img.resize((image_width_px, hsize), Image.LANCZOS)

        # Position the image on the PDF
        img_x = i * int(2 * 300)  # Adjust for overlapping images
        img_y = (pdf_height_px - hsize) // 2

        # Paste the image onto the canvas
        pdf_img.paste(img, (img_x, img_y))

    # Create an ImageDraw object to add text
    draw = ImageDraw.Draw(pdf_img)

    # Specify the title and font properties
    title_text = f'{sub} - {roi}'
    font_size = 25  # Choose your font size
    # Path to the default TrueType font (might vary based on your setup)
    default_font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
    if not os.path.exists(default_font_path):
        default_font_path = "arial.ttf"  # Fallback to Arial if DejaVuSans is not found

    font = ImageFont.truetype(default_font_path, font_size)

    # Specify title position (example: top center)
    title_x = pdf_img.width // 2
    title_y = 60  # adjust this as needed

    # Add text to image
    draw.text((title_x, title_y), title_text, fill="black", font=font, anchor="ms")  # "ms" anchor aligns text middle-center

    # Save the result as a PDF
    pdf_img.save(f'/home/brooke/pacman/across_subject_analyses/subject_tfrs/{sub}_{roi}_all_events.png', "png", resolution=300.0)

In [5]:

def merge_images(roi_or_sub, sub_list, roi_list, output_path):
    
    # roi_or_sub is a region, set the root to be the region
    home_dir = '/home/brooke/pacman/across_subject_analyses/subject_tfrs/'
    if roi_or_sub in roi_list:
        image_path_root = f'{roi_or_sub}_all_events.png'
        image_paths = [f'{home_dir}/{x}_{image_path_root}' for x in sub_list]
    elif roi_or_sub in sub_list:
        image_path_root = 'all_events.png'
        image_paths = [f'{home_dir}/{roi_or_sub}_{x}_{image_path_root}' for x in roi_list]
    
    images = []
    for path in image_paths:
        if os.path.exists(path):
            img = Image.open(path)
            images.append(img)
        # else:
        #     # Create a white square image if the path does not exist
        #     # Assuming default size 100x100, modify as needed
        #     img = Image.new('RGB', (4350, 750), color='white')
        

    widths, heights = zip(*(i.size for i in images))
    total_height = sum(heights)
    max_width = max(widths)

    new_img = Image.new('RGB', (max_width, total_height))

    y_offset = 0
    for img in images:
        new_img.paste(img, (0,y_offset))
        y_offset += img.size[1]

    new_img.save(output_path, "PDF", resolution=300.0)

In [13]:
sub_list = [ "BJH026", "BJH027", "BJH029", "BJH039", "BJH041", "LL19", "LL14", "LL17", "LL10", "LL12", "LL13", "BJH016", "BJH021", "BJH025", "SLCH002"]
roi_list = ['hc', 'amyg', 'ofc', 'cing', 'dacc', 'sgacc', 'dlpfc', 'mfg', 'sfg', 'insula']


## All subs for each region

In [9]:
for region in roi_list:
    for subject in sub_list:
        try:
            print(f'Region: {region} for subject: {subject}')
            create_subject_roi_event_pdf(sub=subject, roi=region)
        except Exception as e:
            print(f"An error occurred with region {region} and subject {subject}: {e}")


Region: hc for subject: BJH026
An error occurred with region hc and subject BJH026: [Errno 2] No such file or directory: '/home/brooke/pacman/preprocessing/BJH026/scripts/figures/BJH026_average_hc_trialonset.png'
Region: amyg for subject: BJH026


/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (158400000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Region: ofc for subject: BJH026
Region: cing for subject: BJH026
Region: dacc for subject: BJH026
An error occurred with region dacc and subject BJH026: [Errno 2] No such file or directory: '/home/brooke/pacman/preprocessing/BJH026/scripts/figures/BJH026_average_dacc_trialonset.png'
Region: sgacc for subject: BJH026
An error occurred with region sgacc and subject BJH026: [Errno 2] No such file or directory: '/home/brooke/pacman/preprocessing/BJH026/scripts/figures/BJH026_average_sgacc_trialonset.png'
Region: dlpfc for subject: BJH026
Region: mfg for subject: BJH026
An error occurred with region mfg and subject BJH026: [Errno 2] No such file or directory: '/home/brooke/pacman/preprocessing/BJH026/scripts/figures/BJH026_average_mfg_trialonset.png'
Region: sfg for subject: BJH026
An error occurred with region sfg and subject BJH026: [Errno 2] No such file or directory: '/home/brooke/pacman/preprocessing/BJH026/scripts/figures/BJH026_average_sfg_trialonset.png'
Region: insula for subject: 

In [15]:
for subject in sub_list:
    merge_images(subject, sub_list, roi_list, f'/home/brooke/pacman/across_subject_analyses/subject_tfrs/{subject}_all_roi_all_events.pdf') 

In [14]:
for roi in roi_list:
    merge_images(roi, sub_list, roi_list, f'/home/brooke/pacman/across_subject_analyses/subject_tfrs/{roi}_all_subs_all_events.pdf') 

In [27]:
sub_list

['BJH026',
 'BJH027',
 'BJH029',
 'BJH039',
 'BJH041',
 'LL19',
 'LL14',
 'LL17',
 'LL10',
 'LL12',
 'LL13',
 'BJH016',
 'BJH021',
 'BJH025',
 'SLCH002']

In [ ]:
/home/brooke/pacman/across_subject_analyses/subject_tfrs/